# VC Copilot Feature Testing Notebook

This notebook allows you to test the VC Copilot backend features directly, including:
1. Website scraping
2. SWOT analysis
3. Deep dive analysis
4. Founder evaluation

Let's start by importing the necessary functions from the backend.

In [ ]:
import sys
import os
import json
from pprint import pprint
import asyncio

# Add the backend directory to the path so we can import from it
sys.path.append(os.path.abspath('.'))

# Import the necessary functions from the backend
from backend.main import scrape_website, analyze_data, run_deep_dive_analysis, evaluate_founders

## 1. Website Scraping

First, let's test the website scraping functionality. This will extract basic information about a startup from its website.

In [ ]:
# Define a function to run async functions in the notebook
async def run_async(coroutine):
    return await coroutine

# Test URLs
test_urls = [
    "https://www.helix-db.com",
    "https://openai.com",
    "https://anthropic.com",
    "https://stripe.com"
]

# Select a URL to test
test_url = test_urls[0]  # Change index to test different URLs
print(f"Scraping data from: {test_url}")

# Scrape website data
scraped_data = await run_async(scrape_website(test_url))

# Display basic information
print(f"Company Name: {scraped_data.company_name}")
print(f"Description: {scraped_data.description[:200]}...")
print(f"Team Info: {scraped_data.team_info}")
print(f"Technologies: {scraped_data.technologies}")
print(f"Social Links: {scraped_data.social_links}")

## 2. SWOT Analysis

Now, let's run a basic SWOT analysis on the scraped data.

In [ ]:
# Run SWOT analysis
analysis_result = await run_async(analyze_data(scraped_data, ["swot"]))

# Display SWOT analysis results
print(f"Company: {analysis_result.company_name}")
print(f"Industry: {analysis_result.industry}")
print(f"Score: {analysis_result.score}/10")
print("\nStrengths:")
for strength in analysis_result.strengths:
    print(f"- {strength}")
    
print("\nWeaknesses:")
for weakness in analysis_result.weaknesses:
    print(f"- {weakness}")
    
print("\nOpportunities:")
for opportunity in analysis_result.opportunities:
    print(f"- {opportunity}")
    
print("\nThreats:")
for threat in analysis_result.threats:
    print(f"- {threat}")
    
print(f"\nRecommendation: {analysis_result.recommendation}")

## 3. Deep Dive Analysis

Let's run a deep dive analysis to get more detailed insights about the company.

In [ ]:
# Run deep dive analysis
deep_dive_result = await run_async(run_deep_dive_analysis(
    scraped_data.company_name, 
    scraped_data.description, 
    scraped_data.raw_text
))

# Display deep dive sections
print("Deep Dive Analysis Sections:")
for section, content in deep_dive_result.items():
    print(f"\n{section.upper()}:")
    if isinstance(content, dict):
        for key, value in content.items():
            if isinstance(value, list):
                print(f"\n{key}:")
                for item in value:
                    print(f"- {item}")
            else:
                print(f"\n{key}: {value}")
    elif isinstance(content, list):
        for item in content:
            print(f"- {item}")
    else:
        print(content)

## 4. Founder Evaluation

Finally, let's test the new founder evaluation feature to predict founder success based on scientific criteria.

In [ ]:
# Run founder evaluation
founder_evaluation = await run_async(evaluate_founders(
    scraped_data.company_name,
    scraped_data.description,
    scraped_data.team_info,
    scraped_data.raw_text,
    deep_dive_result
))

# Display founder evaluation results
print(f"Success Prediction: {founder_evaluation['success_prediction']}")
print(f"\nOverall Assessment: {founder_evaluation['overall_assessment']}")

print("\nEvaluation Criteria:")
for criterion, data in founder_evaluation['evaluation_criteria'].items():
    print(f"\n{criterion.replace('_', ' ').title()}:")
    print(f"Score: {data['score']}/10")
    print(f"Assessment: {data['assessment']}")

## 5. Complete Analysis

Now let's run a complete analysis with all features enabled at once.

In [ ]:
# Run complete analysis
complete_analysis = await run_async(analyze_data(
    scraped_data, 
    ["swot", "deep_dive", "founder_evaluation"]
))

# Convert to JSON for better visualization
analysis_json = json.loads(complete_analysis.json())

# Display basic info
print(f"Company: {analysis_json['company_name']}")
print(f"Industry: {analysis_json['industry']}")
print(f"Score: {analysis_json['score']}/10")
print(f"Analysis Types: {', '.join(analysis_json['analysis_types'])}")

# Save the complete analysis to a file
with open(f"{analysis_json['company_name'].lower()}_analysis.json", "w") as f:
    json.dump(analysis_json, f, indent=2)
    
print(f"\nComplete analysis saved to {analysis_json['company_name'].lower()}_analysis.json")

## 6. Compare Multiple Startups

Let's compare the founder evaluation results for multiple startups.

In [ ]:
async def analyze_startup(url):
    print(f"Analyzing {url}...")
    scraped = await scrape_website(url)
    analysis = await analyze_data(scraped, ["swot", "founder_evaluation"])
    return {
        "company": analysis.company_name,
        "score": analysis.score,
        "success_prediction": analysis.founder_evaluation.get("success_prediction", False) if analysis.founder_evaluation else False,
        "top_strength": analysis.strengths[0] if analysis.strengths else "",
        "top_weakness": analysis.weaknesses[0] if analysis.weaknesses else ""
    }

# Analyze multiple startups
results = []
for url in test_urls:
    result = await run_async(analyze_startup(url))
    results.append(result)
    
# Display comparison table
print("\nStartup Comparison:")
print("-" * 100)
print(f"{'Company':<20} | {'Score':<5} | {'Success':<7} | {'Top Strength':<30} | {'Top Weakness':<30}")
print("-" * 100)
for result in results:
    print(f"{result['company']:<20} | {result['score']:<5} | {str(result['success_prediction']):<7} | {result['top_strength'][:30]:<30} | {result['top_weakness'][:30]:<30}")